In [ ]:
from os import listdir
from os.path import isfile, join
import re
import pandas as pd
from lexnlp.extract.en.entities import nltk_re
from lexnlp.nlp.en.segments import sentences
from lexnlp.utils import parse_df

In [ ]:
DATA_PATH = join('..','data','interim','sfcr')

In [ ]:
txt_files = [f for f in listdir(DATA_PATH) if isfile(join(DATA_PATH, f)) and f[-3:]=='txt']
documents = []
for file_name in txt_files:
    file = open(join(DATA_PATH, file_name), "rb")
    text = file.read().decode('utf-8')
    file.close()
    #text = text.replace("\n", " ")
    documents.append(text)
    
pickle_files = [f for f in listdir(DATA_PATH) if isfile(join(DATA_PATH, f)) and f[-6:]=='pickle']
df = pd.DataFrame()
for file_name in pickle_files:
    df = df.append(pd.read_pickle(join(DATA_PATH, file_name)), ignore_index = True)

In [ ]:
print("Number of documents: " + str(len(documents)))
print("Number of sentences: " + str(len(df.index)))

# Get companies (nltk approach)

In [ ]:
# nltk_re performs a complex regular expression for finding company entities

l = list(nltk_re.get_companies(documents[0]))

In [ ]:
l[0:5]

# Accounting firms

Dutch accounting firms are supervised by the AFM. It publishes a register with accounting firms: https://www.afm.nl/en/professionals/registers/vergunningenregisters/accountantsorganisaties

In [ ]:
# file directly downloaded from AFM website
f = join('..','dictionary','nl','AFM','accountantsorganisaties.csv')
df_accounting = pd.read_csv(f, encoding = 'latin-1', sep= ';').set_index('Naam organisatie')
# additional file for aliases of firm names (for example EY instead of Ernst & Young Accountants)
f = join('..','dictionary','nl','AFM','accountantsorganisaties-alias.csv')
alias = pd.read_csv(f, sep = ',', encoding = 'latin-1').set_index('Naam organisatie')
df_accounting = df_accounting.join(alias).reset_index()

parse_columns = ('Naam organisatie', 'Korte naam', 'Afkorting')
result_columns = {'Naam organisatie': 'name'}
preformed_entity = {'entity_type': 'accounting firm', 
                    'source'     : 'AFM', 
                    'country'    : 'NL'}

In [ ]:
df_results1 = pd.DataFrame()
for idx, document in enumerate(documents):
    for sentence in sentences.get_sentence_list(document):
        items = list(parse_df.DataframeEntityParser(df_accounting, parse_columns, result_columns, preformed_entity).get_entities_from_text(sentence))
        for i in items:
            df_results1 = df_results1.append(pd.DataFrame(columns = ['file name']+list(i.keys())+['text'], 
                                                          data = [[txt_files[idx]]+list(i.values())+[sentence]]), ignore_index = True)

In [ ]:
len(df_results1.index)

In [ ]:
df_results1.head(5)

In [ ]:
df_results2 = pd.DataFrame()
for sent_id in df.index:
    sentence = df.loc[sent_id,'text']
    items = list(parse_df.DataframeEntityParser(df_accounting, parse_columns, result_columns, preformed_entity).get_entities_from_text(sentence))
    for i in items:
        df_results2 = df_results2.append(pd.DataFrame(columns = list(df.loc[sent_id].index) + list(i.keys()), 
                                                      data = [list(df.loc[sent_id].values) + list(i.values())]), ignore_index = True)

In [ ]:
len(df_results2.index)

# Insurance undertakings

EIOPA's register includes all European insurance undertakings.

In [ ]:
f = join('..','dictionary','common','EIOPA','DATINS_Export_637110803393817886.csv')
df_insurers = pd.read_csv(f, encoding = 'latin-1', sep = ';')
df_insurers = df_insurers[df_insurers['Name of NCA']=='De Nederlandsche Bank']

In [ ]:
parse_columns = ('International Name', 'Official name of the entity')
result_columns = {'International Name': 'name', 'Home Country': 'country'}
preformed_entity = {'entity_type': 'insurance undertaking', 
                    'source'     : 'EIOPA'}

In [ ]:
df_results = pd.DataFrame()
for idx, document in enumerate(documents[0:4]):
    for sentence in sentences.get_sentence_list(document):
        items = list(parse_df.DataframeEntityParser(df_insurers, parse_columns, result_columns, preformed_entity).get_entities_from_text(sentence))
        for i in items:
            df_results = df_results.append(pd.DataFrame(columns = ['file name']+list(i.keys())+['sentence'], 
                                                        data = [[txt_files[idx]]+list(i.values())+[str(sentence)]]))
df_results